In [1]:
import numpy as np
import pyuvdata
import newcal
import importlib
importlib.reload(newcal)
from newcal import calibration_optimization
importlib.reload(calibration_optimization)
from newcal import calibration_wrappers
importlib.reload(calibration_wrappers)
from newcal import cost_function_calculations
importlib.reload(cost_function_calculations)
import time
import sympy

In [11]:
data = pyuvdata.UVData()
data.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz.ms")

In [12]:
model = pyuvdata.UVData()
model.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz_model.uvfits")

In [13]:
data.select(freq_chans=[90])
model.select(freq_chans=[90])

In [14]:
(
    gains_init,
    Nants,
    Nbls,
    Ntimes,
    Nfreqs,
    N_feed_pols,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    antenna_names,
) = calibration_wrappers.uvdata_calibration_setup(
    data,
    model,
    gain_init_calfile=None,
    gain_init_stddev=0.0,
    N_feed_pols=2,
    min_cal_baseline=30,
)

divide by zero encountered in true_divide


In [35]:
start_time = time.time()
gains_fit = calibration_wrappers.calibration_per_pol(
    gains_init,
    Nants,
    Nbls,
    Nfreqs,
    N_feed_pols,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
    xtol=1e-3,
    parallel=False,
    verbose=True,
)
print(time.time() - start_time)

Optimization terminated successfully.
         Current function value: 814791605889.980713
         Iterations: 48
         Function evaluations: 68
         Gradient evaluations: 68
         Hessian evaluations: 48
Optimization terminated successfully.
Optimization time: 0.8184708674748739 minutes
Optimization terminated successfully.
         Current function value: 655482456364.638672
         Iterations: 31
         Function evaluations: 36
         Gradient evaluations: 36
         Hessian evaluations: 31
Optimization terminated successfully.
Optimization time: 0.46125821272532147 minutes
Optimization time: 1 frequency channels in 1.307828466097514 minutes
78.47061896324158


In [7]:
initial_cost = cost_function_calculations.cost_function_single_pol(
    gains_init[:, 0, 0],
    model_visibilities[:, :, 0, 0],
    data_visibilities[:, :, 0, 0],
    visibility_weights[:, :, 0, 0],
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
)
print(initial_cost)

17159110543298.645


In [29]:
gains_init_flattened = np.stack(
    (np.real(gains_init[:,0,0]), np.imag(gains_init[:,0,0])),
    axis=0,
).flatten()
start_time = time.time()
cost = calibration_optimization.cost_function_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, 0, 0],
    data_visibilities[:, :, 0, 0],
    visibility_weights[:, :, 0, 0],
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
)
print(time.time() - start_time)

0.2766449451446533


In [32]:
start_time = time.time()
jac = calibration_optimization.jacobian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, 0, 0],
    data_visibilities[:, :, 0, 0],
    visibility_weights[:, :, 0, 0],
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
)
print(time.time() - start_time)

0.6730918884277344


In [33]:
start_time = time.time()
hess = calibration_optimization.hessian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, 0, 0],
    data_visibilities[:, :, 0, 0],
    visibility_weights[:, :, 0, 0],
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
)
print(time.time() - start_time)

0.6498711109161377


In [3]:
sympy.Matrix([[2],[4]]).rref()

(Matrix([
 [1],
 [0]]),
 (0,))